<a href="https://colab.research.google.com/github/JulioSan02/UNCC/blob/4105/HW5/JS_4105_HW5_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/Data/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [35]:
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def data_map(x):
    return x.map({'yes': 1, 'no': 0, 'unfurnished': 0, 'semi-furnished': 0.5, 'furnished': 1})

# Applying the function to the housing list
housing[varlist] = housing[varlist].apply(data_map)
print('All housing data with binary inputs replaced with ones and zeros: \n')
housing.head()

All housing data with binary inputs replaced with ones and zeros: 



,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1.0,0.0,0.0,0.0,1.0,2,1.0,1.0
1,12250000,8960,4,4,4,1.0,0.0,0.0,0.0,1.0,3,0.0,1.0
2,12250000,9960,3,2,2,1.0,0.0,1.0,0.0,0.0,2,1.0,0.5
3,12215000,7500,4,2,2,1.0,0.0,1.0,0.0,1.0,3,1.0,1.0
4,11410000,7420,4,1,2,1.0,1.0,1.0,0.0,1.0,2,0.0,1.0


In [36]:
np.random.seed(0)
train, test = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 100)

scale = StandardScaler()

price_train_og = train.values[:, 0]
price_train_t = torch.tensor(price_train_og)

price_test_og = test.values[:, 0]
price_test_t = torch.tensor(price_test_og)

train_len = len(price_train_og)  # Number of training examples for training set
test_len = len(price_test_og)  # Number of training examples for test set

vars_stand  = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

train_stand_og = train
train_stand_og[vars_stand] = scale.fit_transform(train[vars_stand])

test_stand_og = test
test_stand_og[vars_stand] = scale.fit_transform(test[vars_stand])

# Inputs using only 5 variables
vars_5_train_stand_og = np.c_[np.ones((train_len, 1)), train_stand_og[vars_stand]]
vars_5_train_stand = torch.tensor(vars_5_train_stand_og)

vars_5_test_stand_og = np.c_[np.ones((test_len, 1)), test_stand_og[vars_stand]]
vars_5_test_stand = torch.tensor(vars_5_test_stand_og)


params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)


In [37]:
def model(t_in, w1, w2, w3, w4, w5, b):
  t_p = w5 * t_in ** 5 + w4 * t_in ** 4 + w3 * t_in ** 3 + w2 * t_in ** 2 + w1 * t_in + b
  return t_p

def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c)**2
  return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, train_t_in, test_t_in,
                    train_t_out, test_t_out):
  for epoch in range(1, n_epochs + 1):

    if params.grad is not None:
      params.grad.zero_()

    train_t_p = model(train_t_in, *params)
    train_loss = loss_fn(train_t_p.transpose(0,1), train_t_out)

    test_t_p = model(test_t_in, *params)
    test_loss = loss_fn(test_t_p.transpose(0,1), test_t_out)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
      print(f"Epoch {epoch}, \tTraining loss {train_loss.item():.4f},"
      f" \tValidation loss {test_loss.item():.4f}")

  return params

In [38]:
learning_rate_1 = 0.00001
optimizer_1 = optim.SGD([params_og], lr=learning_rate_1)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_1,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 26477367215968.7031, 	Validation loss 25158550922316.9062
Epoch 2, 	Training loss 26252492095871.8359, 	Validation loss 24945519050921.7812
Epoch 3, 	Training loss 26148701010961.7305, 	Validation loss 24829291114436.1719
Epoch 500, 	Training loss 24030485177568.9609, 	Validation loss 22646973184094.4531
Epoch 1000, 	Training loss 22780481948549.3828, 	Validation loss 21415103917087.7383
Epoch 1500, 	Training loss 21935999137838.0742, 	Validation loss 20592738554701.4453
Epoch 2000, 	Training loss 21298742665618.6797, 	Validation loss 19979624402917.0820
Epoch 2500, 	Training loss 20770003114723.2617, 	Validation loss 19475899807582.8242
Epoch 3000, 	Training loss 20300576152408.7539, 	Validation loss 19031684456492.0938
Epoch 3500, 	Training loss 19865865039011.7891, 	Validation loss 18621989617610.3672
Epoch 4000, 	Training loss 19453507484511.2461, 	Validation loss 18234231820893.9219
Epoch 4500, 	Training loss 19057228128575.9688, 	Validation loss 1786201466

tensor([ 66685.3047, 286467.4062, 106032.3125, 176000.3125, -38740.5781,
        409589.6562], requires_grad=True)

In [39]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_2 = 0.000001
optimizer_2 = optim.SGD([params_og], lr=learning_rate_2)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_2,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 26477367215968.7031, 	Validation loss 25158550922316.9062
Epoch 2, 	Training loss 26450970823793.1367, 	Validation loss 25134971074532.6055
Epoch 3, 	Training loss 26426254803436.0586, 	Validation loss 25112631721337.1133
Epoch 500, 	Training loss 25820276413173.9805, 	Validation loss 24426579742799.2422
Epoch 1000, 	Training loss 25575079580189.0156, 	Validation loss 24182119047036.0664
Epoch 1500, 	Training loss 25343292321509.7305, 	Validation loss 23951181272940.2812
Epoch 2000, 	Training loss 25124017140510.2344, 	Validation loss 23732862555766.1914
Epoch 2500, 	Training loss 24916417849139.0742, 	Validation loss 23526320883676.2188
Epoch 3000, 	Training loss 24719714134902.9531, 	Validation loss 23330770043072.5430
Epoch 3500, 	Training loss 24533178899507.9258, 	Validation loss 23145477743344.2461
Epoch 4000, 	Training loss 24356133660233.6953, 	Validation loss 22969760345724.5000
Epoch 4500, 	Training loss 24187945435336.1055, 	Validation loss 2280298009

tensor([ 11566.9668,  42781.8594,  34186.9023,  69291.5156, -11096.2314,
         46328.5703], requires_grad=True)

In [40]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_3 = 0.0000001
optimizer_3 = optim.SGD([params_og], lr=learning_rate_3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_3,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 26477367215968.7031, 	Validation loss 25158550922316.9062
Epoch 2, 	Training loss 26474688488070.5859, 	Validation loss 25156170171039.0859
Epoch 3, 	Training loss 26472027075586.1562, 	Validation loss 25153802121501.6133
Epoch 500, 	Training loss 26067387596002.9102, 	Validation loss 24713083342159.1133
Epoch 1000, 	Training loss 26026794648863.9219, 	Validation loss 24641078354704.7539
Epoch 1500, 	Training loss 25999954231592.3789, 	Validation loss 24607449629622.0898
Epoch 2000, 	Training loss 25973761914003.5195, 	Validation loss 24579991653993.3438
Epoch 2500, 	Training loss 25947737491102.3047, 	Validation loss 24553776987536.0742
Epoch 3000, 	Training loss 25921862421674.1758, 	Validation loss 24527916889079.1484
Epoch 3500, 	Training loss 25896135141619.9102, 	Validation loss 24502243883404.7891
Epoch 4000, 	Training loss 25870554452435.6406, 	Validation loss 24476726253674.5938
Epoch 4500, 	Training loss 25845119391821.5391, 	Validation loss 2445135688

tensor([1282.4481, 4644.8652, 4177.6333, 9066.3311, 2931.9880, 4736.5200],
       requires_grad=True)

In [41]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_4 = 0.00000001
optimizer_4 = optim.SGD([params_og], lr=learning_rate_4)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_4,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 26477367215968.7031, 	Validation loss 25158550922316.9062
Epoch 2, 	Training loss 26477098952278.0352, 	Validation loss 25158312619512.0352
Epoch 3, 	Training loss 26476830862293.8867, 	Validation loss 25158074444055.5430
Epoch 500, 	Training loss 26362902642205.5742, 	Validation loss 25054020441409.7812
Epoch 1000, 	Training loss 26279699032931.1758, 	Validation loss 24973195836521.5195
Epoch 1500, 	Training loss 26219154730831.5547, 	Validation loss 24910312997355.1562
Epoch 2000, 	Training loss 26174895242485.1016, 	Validation loss 24860945736125.0703
Epoch 2500, 	Training loss 26142339432181.3984, 	Validation loss 24821817843004.7188
Epoch 3000, 	Training loss 26118195052673.3203, 	Validation loss 24790494245740.6250
Epoch 3500, 	Training loss 26100095970774.1836, 	Validation loss 24765156508665.1211
Epoch 4000, 	Training loss 26086341871989.9141, 	Validation loss 24744440151036.5312
Epoch 4500, 	Training loss 26075710824583.3164, 	Validation loss 2472731569

tensor([ 138.5986,  501.9268,  573.3611, 1616.7448, 3694.6321,  476.8972],
       requires_grad=True)

In [42]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_5 = 0.0001
optimizer_5 = optim.Adam([params_og], lr=learning_rate_5)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_5,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 26477367215968.7031, 	Validation loss 25158550922316.9062
Epoch 2, 	Training loss 26477367191749.3750, 	Validation loss 25158550900034.1211
Epoch 3, 	Training loss 26477367167530.0508, 	Validation loss 25158550877751.3320
Epoch 500, 	Training loss 26477355130590.4727, 	Validation loss 25158539803249.9102
Epoch 1000, 	Training loss 26477343021108.0781, 	Validation loss 25158528661967.6133
Epoch 1500, 	Training loss 26477330911750.5664, 	Validation loss 25158517520762.2070
Epoch 2000, 	Training loss 26477318802519.1758, 	Validation loss 25158506379634.8984
Epoch 2500, 	Training loss 26477306693413.9102, 	Validation loss 25158495238585.7070
Epoch 3000, 	Training loss 26477294584506.0781, 	Validation loss 25158484097683.7773
Epoch 3500, 	Training loss 26477282475724.5273, 	Validation loss 25158472956860.0898
Epoch 4000, 	Training loss 26477270367069.0898, 	Validation loss 25158461816114.5000
Epoch 4500, 	Training loss 26477258258539.7812, 	Validation loss 2515845067

tensor([1.5001, 1.5001, 1.5001, 1.5001, 1.5001, 0.5000], requires_grad=True)

In [43]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_6 = 0.001
optimizer_6 = optim.Adam([params_og], lr=learning_rate_6)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_6,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 26477367215968.7031, 	Validation loss 25158550922316.9062
Epoch 2, 	Training loss 26477366973803.1602, 	Validation loss 25158550699514.5195
Epoch 3, 	Training loss 26477366731637.6758, 	Validation loss 25158550476712.1602
Epoch 500, 	Training loss 26477246391130.5781, 	Validation loss 25158439756431.1133
Epoch 1000, 	Training loss 26477125340954.8789, 	Validation loss 25158328379506.2266
Epoch 1500, 	Training loss 26477004303301.6562, 	Validation loss 25158217010314.6250
Epoch 2000, 	Training loss 26476883278257.6797, 	Validation loss 25158105648932.5039
Epoch 2500, 	Training loss 26476762275431.2305, 	Validation loss 25157994303836.4297
Epoch 3000, 	Training loss 26476641294619.6094, 	Validation loss 25157882974796.8242
Epoch 3500, 	Training loss 26476520308030.7188, 	Validation loss 25157771637445.9844
Epoch 4000, 	Training loss 26476399333976.9844, 	Validation loss 25157660307839.6953
Epoch 4500, 	Training loss 26476278372527.3477, 	Validation loss 2515754898

tensor([5.9997, 5.9997, 5.9997, 5.9997, 5.9995, 4.9998], requires_grad=True)

In [44]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_7 = 0.01
optimizer_7 = optim.Adam([params_og], lr=learning_rate_7)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_7,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 26477367215968.7031, 	Validation loss 25158550922316.9062
Epoch 2, 	Training loss 26477364794426.4727, 	Validation loss 25158548694396.2500
Epoch 3, 	Training loss 26477362372889.2734, 	Validation loss 25158546466478.6953
Epoch 500, 	Training loss 26476159766895.7148, 	Validation loss 25157439824976.5273
Epoch 1000, 	Training loss 26474951683943.3633, 	Validation loss 25156327756017.8828
Epoch 1500, 	Training loss 26473745285137.2539, 	Validation loss 25155216850332.9648
Epoch 2000, 	Training loss 26472540478647.7930, 	Validation loss 25154107025189.2031
Epoch 2500, 	Training loss 26471337193633.3086, 	Validation loss 25152998218622.4727
Epoch 3000, 	Training loss 26470135354330.6758, 	Validation loss 25151890359623.5664
Epoch 3500, 	Training loss 26468934850788.1758, 	Validation loss 25150783348623.9648
Epoch 4000, 	Training loss 26467735890521.0859, 	Validation loss 25149677375451.2617
Epoch 4500, 	Training loss 26466538207838.4258, 	Validation loss 2514857220

tensor([50.9908, 50.9879, 50.9609, 50.9423, 50.9127, 49.9974],
       requires_grad=True)

In [45]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_8 = 0.1
optimizer_8 = optim.Adam([params_og], lr=learning_rate_8)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_8,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 26477367215968.7031, 	Validation loss 25158550922316.9062
Epoch 2, 	Training loss 26477343000745.6133, 	Validation loss 25158528643225.4062
Epoch 3, 	Training loss 26477318786051.1406, 	Validation loss 25158506364468.1211
Epoch 500, 	Training loss 26465373641504.7227, 	Validation loss 25147497038811.4258
Epoch 1000, 	Training loss 26453533106200.8203, 	Validation loss 25136545635079.7109
Epoch 1500, 	Training loss 26441859035500.5039, 	Validation loss 25125709707442.9180
Epoch 2000, 	Training loss 26430341830218.5195, 	Validation loss 25114980928383.0195
Epoch 2500, 	Training loss 26418973361440.1016, 	Validation loss 25104352200308.8516
Epoch 3000, 	Training loss 26407747047229.5273, 	Validation loss 25093817748756.0117
Epoch 3500, 	Training loss 26396657638961.6680, 	Validation loss 25083372951985.1016
Epoch 4000, 	Training loss 26385701061359.8281, 	Validation loss 25073014200595.6094
Epoch 4500, 	Training loss 26374874399777.7031, 	Validation loss 2506273892

tensor([499.8086, 499.7374, 496.9658, 495.2081, 491.9282, 499.8973],
       requires_grad=True)